In [1]:
import pandas as pd
import re

from fluxo.dados import Dados
from fluxo.carga_dados import CargaDados
from fluxo.limpeza_dados import LimpezaDados

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
tipo_ato = 'SC'
dados = Dados()
cg = CargaDados()
cg.executar(dados)

lp = LimpezaDados(dados)
lp.executar(dados, tipo_ato, reclassificar_nao_identificados=True)

16:04:39 - Carregando dados de segmentos...
16:04:39 - 206488 registros carregados.
16:04:39 - (Re)Carregando segmentos originais...
16:04:39 - Removendo segmentos nulos...
16:04:39 - 1900 segmentos nulos excluídos.
16:04:39 - Restaram 204588 segmentos não nulos.
16:04:39 - Removendo todos os segmentos do tipo Anexo...
16:04:40 - 5771 segmentos do tipo Anexo excluídos.
16:04:40 - Determinando tipos de segmento (labels) para atos do tipo SC...
16:04:40 - Tipos de segmento para atos SC: ['Ementa', 'Fecho', 'Não Identificado']
16:04:40 - Excluindo segmentos dos atos que não são SC...
16:04:40 - 188070 segmentos de atos não SC excluídos.
16:04:40 - Restaram 10747 segmentos de atos SC.
16:04:40 - Removendo segmentos não representativos para o tipo SC...
16:04:40 - 100 segmentos não representativos excluídos.
16:04:40 - Restaram 10647 segmentos representativos.
16:04:40 - Removendo tags HTML...
16:04:40 - Removendo caracteres de escape HTML...
16:04:40 - Reclassificando segmentos não identif

In [3]:
df = dados.limp
#df[df['tipo_seg']=='Artigo'].sample(10)

In [4]:
def classificar_por_padroes(texto_segmento):

    padrao = {
        'Artigo'    : '^"{0,1}\s*(Art|ART)\.\s*',
        'Alínea'    : '^(\s*)([a-z]\))',
        'Inciso'    : '^(\s*)M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})(\s-|\s–)',
        'Parágrafo' : '^(\s*)(§|Parágrafo|Paragrafo)'
    }

    for label in dados.labels:
        if label in padrao.keys():
            if re.match(padrao[label], str(texto_segmento)): 
                return label
       
    return 'Outro'

In [5]:
df['tipo_seg_regex'] = df['txt_seg'].apply(classificar_por_padroes)

In [6]:
#df[(df['tipo_seg']!=df['tipo_seg_regex'])&(df['tipo_seg']=='Inciso')]

In [7]:
#df['tipo_seg'].value_counts()
#df['tipo_seg_regex'].value_counts()

In [8]:
#padrao = '^(\s*)M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})(\s-|\s–)'
#df[df['txt_seg'].str.contains(padrao)]['id_seg'].count()

## Reclassificar como **Outro** os segmentos de atos não classificados por REGEX

In [9]:
df.loc[~df['tipo_seg'].isin(['Artigo', 'Alínea', 'Parágrafo', 'Inciso']), 'tipo_seg'] = 'Outro'

In [10]:
df['tipo_seg_regex'].unique()

array(['Outro'], dtype=object)

## Preparar conjunto real e predito

In [11]:
y = df['tipo_seg'].to_list()
y_pred = df['tipo_seg_regex'].to_list()

In [12]:
precision = round(precision_score(y, y_pred, average='macro'),4)             
recall = round(recall_score(y, y_pred, average='macro'),4)
f1 = round(f1_score(y, y_pred, average='macro'),4)
accuracy = round(accuracy_score(y, y_pred),4)

In [13]:
print(f'Acurácia: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Acurácia: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
